# 1. Setup

In [ ]:
# import json
# from datetime import date, time

# import colorcet as cc
# import holidays
# import matplotlib.patches as mpatches
# import matplotlib.pyplot as plt
# import matplotlib.transforms as mt
import numpy as np
import pandas as pd
# import plotly.express as px
# import plotly.graph_objects as go
# import seaborn as sns
# from IPython.display import Image
# from matplotlib.dates import DateFormatter, MonthLocator
# from matplotlib.lines import Line2D
# from matplotlib.ticker import MultipleLocator
# from pandas.core.groupby.generic import DataFrameGroupBy
# from prophet import Prophet
# from scipy.ndimage import binary_dilation
# from shapely.geometry import shape
# from statsmodels.tsa.seasonal import MSTL, seasonal_decompose

In [ ]:
RAND = 10
RAW_DATA_PATH = "../raw_data/"
ADDITIONAL_DATA_PATH = "../additional_data/"

In [ ]:
from utils.load import load_all_raw_data
from utils.process import process_original_dfs
from utils.process import process_additional_dfs
from utils.merge import merge_all_dfs

In [ ]:
original_data, additional_data = load_all_raw_data(
    RAW_DATA_PATH, ADDITIONAL_DATA_PATH
)
df = merge_all_dfs(
    process_original_dfs(original_data),
    process_additional_dfs(additional_data),
)

In [ ]:
df.shape

(2012370, 43)

In [ ]:
df

,county,product_type,is_business,is_consumption,datetime,target,data_block_id,date,eic_count,installed_capacity,...,h2_cloudcover_high,h2_cloudcover_total,h2_windspeed,h2_u_component,h2_v_component,h2_shortwave_radiation,h2_direct_solar_radiation,h2_diffuse_radiation,dst,holiday_type
0,0,1,0,0,2021-09-03 00:00:00,0.793000,2,2021-09-03,108,952.890015,...,4,59,4.972222,1.492195,-4.671064,0,0,0,True,ordinary_day
1,0,1,0,1,2021-09-03 00:00:00,107.128998,2,2021-09-03,108,952.890015,...,4,59,4.972222,1.492195,-4.671064,0,0,0,True,ordinary_day
2,0,2,0,0,2021-09-03 00:00:00,0.000000,2,2021-09-03,17,166.399994,...,4,59,4.972222,1.492195,-4.671064,0,0,0,True,ordinary_day
3,0,2,0,1,2021-09-03 00:00:00,19.629999,2,2021-09-03,17,166.399994,...,4,59,4.972222,1.492195,-4.671064,0,0,0,True,ordinary_day
4,0,3,0,0,2021-09-03 00:00:00,0.977000,2,2021-09-03,688,7207.879883,...,4,59,4.972222,1.492195,-4.671064,0,0,0,True,ordinary_day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012365,15,0,1,1,2023-05-31 23:00:00,197.233002,637,2023-05-31,15,620.000000,...,30,21,1.772222,0.802269,-0.718775,0,0,0,True,ordinary_day
2012366,15,1,1,0,2023-05-31 23:00:00,0.000000,637,2023-05-31,20,624.500000,...,30,21,1.772222,0.802269,-0.718775,0,0,0,True,ordinary_day
2012367,15,1,1,1,2023-05-31 23:00:00,28.403999,637,2023-05-31,20,624.500000,...,30,21,1.772222,0.802269,-0.718775,0,0,0,True,ordinary_day
2012368,15,3,1,0,2023-05-31 23:00:00,0.000000,637,2023-05-31,55,2188.199951,...,30,21,1.772222,0.802269,-0.718775,0,0,0,True,ordinary_day


In [ ]:
pd.DataFrame({"date": pd.date_range(df.datetime.min(),df.datetime.max(), freq='D'), "holiday_type": "ordinary_day"})

,date,holiday_type
0,2021-09-03,ordinary_day
1,2021-09-04,ordinary_day
2,2021-09-05,ordinary_day
3,2021-09-06,ordinary_day
4,2021-09-07,ordinary_day
...,...,...
631,2023-05-27,ordinary_day
632,2023-05-28,ordinary_day
633,2023-05-29,ordinary_day
634,2023-05-30,ordinary_day


In [ ]:
df.columns

Index(['county', 'product_type', 'is_business', 'is_consumption', 'datetime',
       'target', 'data_block_id', 'date', 'eic_count', 'installed_capacity',
       'lowest_price_per_mwh', 'highest_price_per_mwh', 'euros_per_mwh',
       'f1_temperature', 'f1_dewpoint', 'f1_snowfall',
       'f1_total_precipitation', 'f1_cloudcover_low', 'f1_cloudcover_mid',
       'f1_cloudcover_high', 'f1_cloudcover_total', 'f1_u_component',
       'f1_v_component', 'f1_windspeed', 'f1_direct_solar_radiation',
       'f1_surface_solar_radiation_downwards', 'h2_temperature', 'h2_dewpoint',
       'h2_rain', 'h2_snowfall', 'h2_surface_pressure', 'h2_cloudcover_low',
       'h2_cloudcover_mid', 'h2_cloudcover_high', 'h2_cloudcover_total',
       'h2_windspeed', 'h2_u_component', 'h2_v_component',
       'h2_shortwave_radiation', 'h2_direct_solar_radiation',
       'h2_diffuse_radiation', 'dst', 'holiday_type'],
      dtype='object')

In [ ]:
qwe

NameError: name 'qwe' is not defined

In [ ]:
# df.loc[df.dateisna().sum()'2023-03-26 03:00:00'

In [ ]:
CATEGORICAL_DICT = {
    "county": county_id_to_name_map,
    "is_business": {0: "not_business", 1: "business"},
    "is_consumption": {0: "production", 1: "consumption"},
    "product_type": {
        0: "combined",
        1: "fixed",
        2: "general_service",
        3: "spot",
    },
}

PALETTE = sns.color_palette(
    cc.glasbey[:4]
    + [cc.glasbey[8]]
    + cc.glasbey[5:8]
    + [cc.glasbey[4]]
    + [cc.glasbey[12]]
    + cc.glasbey[10:12]
    + [cc.glasbey[9]]
    + cc.glasbey[13:16]
).as_hex()

SEGMENT_C = ["county", "product_type", "is_business"]
CATEGORICAL_C = ["county", "product_type", "is_business", "is_consumption"]
TARGET_C = [
    "county",
    "product_type",
    "is_business",
    "is_consumption",
    "datetime",
]

In [ ]:
pd.set_option(
    "display.float_format",
    lambda x: f"{x:.2e}" if abs(x) < 0.01 and x != 0 else f"{x:.2f}",
)
pd.set_option("display.max_columns", None)

In [ ]:
sns.set_style("whitegrid")